In [1]:
import pandas as pd
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import lars_path, LinearRegression, Lasso, LassoCV
from sklearn.metrics import r2_score, mean_squared_error
import scipy.stats as stats
import matplotlib.pyplot as plt
import math
import requests, io

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import partial_dependence, permutation_importance, PartialDependenceDisplay
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

%matplotlib inline

In [48]:
#url = "https://raw.githubusercontent.com/Jacopo21/HM2/main/listings_APR_FINAL.csv?token=GHSAT0AAAAAACJM5W7SSV47RFU7DTAMJELYZLA53VA"
#download = requests.get(url).content
data_apr = pd.read_csv("/Users/Admin/Desktop/listings_APR_FINAL.csv")
data_apr.head()

,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,...,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,ln_price,property_House,property_Private room,property_Shared Room
0,100%,100%,f,2.0,5.0,"['email', 'phone']",t,t,Canarsie,Brooklyn,...,4.94,2,1,1,0,4.46,3.401197,False,True,False
1,NaN,NaN,f,1.0,5.0,"['email', 'phone']",t,t,Civic Center,Manhattan,...,NaN,1,0,1,0,0.01,3.951244,False,True,False
2,NaN,NaN,f,1.0,1.0,"['email', 'phone']",t,t,Greenpoint,Brooklyn,...,5.00,1,0,1,0,0.11,4.787492,False,True,False
3,NaN,NaN,f,1.0,1.0,"['email', 'phone']",t,f,Upper West Side,Manhattan,...,4.90,1,1,0,0,0.12,5.010635,True,False,False
4,NaN,NaN,f,1.0,1.0,"['email', 'phone']",t,t,Washington Heights,Manhattan,...,5.00,1,0,1,0,0.01,3.218876,False,True,False


In [ ]:
variable_names = data_apr.columns.tolist()
variable_table = pd.DataFrame(variable_names, columns=["Variable Names"])
pd.set_option('display.max_rows', None)
print(variable_table)

In [49]:
# Remove various strings, split amenities and turn into dtype:object
data_apr["amenities"] = data_apr["amenities"].str.strip("[]").str.replace('"', "").str.split(",")

In [41]:
# Function to merge amenities containing any keyword from a list (case-insensitive)
def merge_items_with_keywords(amenities_list, keywords):
    merged_amenities = []
    for amenities in amenities_list:
        merged_item = []
        for amenity in amenities:
            lower_amenity = amenity.lower()
            if any(keyword in lower_amenity for keyword in keywords):
                merged_item.append(next(keyword for keyword in keywords if keyword in lower_amenity))
            else:
                merged_item.append(amenity)
        merged_amenities.append(list(set(merged_item)))
    return merged_amenities

In [50]:
keyword_list=['body', 'shampoo', 'conditioner', 'wifi', 'sound system', 
              'tv', 'stove', 'oven', 'microwave', 'refrigerator', 
              'dishwasher', 'hair dryer', 'dryer', 'kettle', 'toaster', 
              'baby bath', 'coffee maker', 'game console', 'toys', 'breakfast', 
              'clothing storage', 'washer', 'bbc grill', 'backyard']

In [51]:
def merge_items_with_keywords(amenities_list, merge_dict):
    merged_amenities = []
    for amenities in amenities_list:
        merged_item = []
        for amenity in amenities:
            lower_amenity = amenity.lower()
            for new_category, old_categories in merge_dict.items():
                if any(old_category in lower_amenity for old_category in old_categories):
                    merged_item.append(new_category)
                    break
            else:
                merged_item.append(amenity)
        merged_amenities.append(list(set(merged_item)))
    return merged_amenities

In [54]:
merge_dict = {
    'wifi': ['wifi'],
    'kitchen': ['kitchen', ' kitchen', 'kitchenette', ' kitchenette'],
    'stove': ['stove'],
    'oven': ['oven'],
    'microvawe': ['microwave'],
    'refrigerator': ['refrigerator', 'fridge'],
    'dishwasher': ['dishwasher'],
    'kettel': ['kettle'],
    'toaster': ['toaster'],
    'coffee': ['coffee maker', 'machine', 'coffee', 'espresso', 'nespresso'],
    'tv': ['tv'],
    'sound_system': ['speaker', 'sound'],
    'game_console': ['game console', 'ps', 'xbox'],
    'baby': ['baby', 'toys'],
    'body_wash': ['body', 'soap', 'shower_gel'],
    'shampoo': ['shampoo'],
    'conditioner': ['conditioner'],
    'hair dryer': ['hair dryer'],
    'laundry': ['washer', 'laundry'],
    'backyard': ['backyard'],
    'grill': ['grill'],
    'breakfast': ['breakfast'],
    'clothing_storage': ['clothing storage'],
    'ac': ['ac - split type ductless system', 'air conditioning', 'central air conditioning', 'window ac unit'],
    'heating': ['heating']
}

In [ ]:
data_apr["amenities"]

In [55]:
# generate dummies by amenities
#print(sorted(data_apr["amenities"].explode().unique()))
data_apr['amenities'] = merge_items_with_keywords(data_apr['amenities'], merge_dict)
#dummies = pd.get_dummies(data_apr.amenities.apply(pd.Series).stack()).sum(level=0)
data_apr['amenities'].head()


0    [ First aid kit, kitchen, tv,  Smoke alarm,  F...
1    [hair dryer,  Iron, kitchen, shampoo, tv,  Ess...
2    [ Lock on bedroom door, hair dryer,  Dryer,  F...
3    [stove, oven,  Cooking basics,  Bathtub, micro...
4    [ Lock on bedroom door, kitchen, shampoo,  Smo...
Name: amenities, dtype: object

In [56]:
dummies = data_apr['amenities'].str.join('|').str.get_dummies()
#dummies = data_apr['amenities'].explode().str.get_dummies()

In [57]:
dummies.head()

,R&Co,100% Vegan,1802 Beekman toiletries,2-5 years old,24-hour fitness center,26.4 QT/25 L,365,5-10 years old,AVON,Acca Kappa toiletries,...,laundry,microvawe,oven,refrigerator,shampoo,sound_system,stove,toaster,tv,wifi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,1
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,1
3,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,0,1,0,1,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


In [47]:
top = dummies.sum().nlargest(100)
print(top)

wifi                                     42267
 Smoke alarm                             38516
 Kitchen                                 38044
 Essentials                              35022
tv                                       33400
                                         ...  
 Coffee maker: Keurig coffee machine      1647
 Private backyard \u2013 Fully fenced     1646
 Paid washer \u2013 In building           1631
 Pack \u2019n play/Travel crib            1587
 HBO Max                                  1565
Length: 100, dtype: int64


In [ ]:
print(dummies.columns)

In [ ]:
# add new numeric columns from certain columns

numericals = [
    "accommodates",
    "bathrooms",
    "review_scores_rating",
    "number_of_reviews",
    "guests_included",
    "reviews_per_month",
    "extra_people",
    "minimum_nights",
    "beds",
]

for col in numericals:
    data_apr["n_" + col] = pd.to_numeric(data_apr[col], errors="coerce")

In [ ]:
# Basic Variables inc neighnourhood
basic_vars = [
    "n_accommodates",
    "n_beds",
    "n_days_since",
    "f_property_type",
    "f_room_type",
    "f_bathroom",
    "f_cancellation_policy",
    "f_bed_type",
    "f_neighbourhood_cleansed",
]

# reviews
reviews = [
    "n_number_of_reviews",
    "flag_n_number_of_reviews",
    "n_review_scores_rating",
    "flag_review_scores_rating",
]

# Dummy variables
amenities = [col for col in data if col.startswith("d_")]

# interactions for the LASSO
# from ch14
X1 = [
    "n_accommodates:f_property_type",
    "f_room_type:f_property_type",
    "f_room_type:d_familykidfriendly",
    "d_airconditioning:f_property_type",
    "d_cats:f_property_type",
    "d_dogs:f_property_type",
]
# with boroughs
X2 = [
    "f_property_type:f_neighbourhood_cleansed",
    "f_room_type:f_neighbourhood_cleansed",
    "n_accommodates:f_neighbourhood_cleansed",
]

In [ ]:
#Split the data
data_train, data_holdout = train_test_split(data, train_size=0.8, random_state=42)

# OLS

In [ ]:
y, X = dmatrices("price ~ " + " + ".join(predictors_2), data_train)

ols_model = LinearRegression().fit(X,y)

#y_test, X_test = dmatrices("price ~ " + " + ".join(predictors_2), data_holdout)

y_hat = ols_model.predict(X)

ols_rmse = mean_squared_error(y,y_hat,squared=False)
ols_rmse

In [ ]:
ols_model_coeffs_df = pd.DataFrame(
    ols_model.coef_.tolist()[0],
    index=X.design_info.column_names,
    columns=["ols_coefficient"],
).assign(ols_coefficient=lambda x: x.ols_coefficient.round(3))

In [ ]:
ols_model_coeffs_df

# LASSO

In [ ]:
from sklearn.linear_model import ElasticNet

In [ ]:
lasso_model_cv = GridSearchCV(
    lasso_model,
    {"alpha": [i / 100 for i in range(1, 26, 1)]},
    cv=5,
    scoring="neg_root_mean_squared_error",
    verbose=3,
)


In [ ]:
y, X = dmatrices("price ~ " + " + ".join(predictors_E), data_train)

In [ ]:
lasso_model_cv.fit(X, y.ravel())